In [93]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv
import torch.utils.data

from sklearn.mixture import GaussianMixture 
from sklearn.model_selection import train_test_split
from torch.utils.data import random_split

import matplotlib.pyplot as plt
%matplotlib inline

In [94]:
def loadData(file):
    data = pd.read_csv(file)
    print('Raw shape: ',data.shape)
    data['date'] = pd.to_datetime(data.date)
    print('Days: ',len(set(data.date)))
    return data

In [95]:
def getTimeSeries(df):
    table = pd.pivot_table(df, values='amount', index=['date'],
                    columns=['start_id','end_id'], aggfunc=np.sum, fill_value=0)
    return table,table.index.values

In [96]:
def getMatrixData():
    file = dataDir + dataFile
    dataRaw = loadData(file)
    dataTs,dates = getTimeSeries(dataRaw)
    matrix = dataTs.values
    print('Matrix Shape: ',matrix.shape)
    return matrix,dates

In [97]:
dataDir = '/home/urwa/Documents/Projects/AnomalyDetection/Pipeline/data/'
dataFile = '20190402_TaipeiEdgesDatewise.csv'
events_data =dataDir+'TaipeiEvents.csv'

In [98]:
file = dataDir + dataFile
dataRaw = loadData(file)

Raw shape:  (7374816, 5)
Days:  638


In [99]:
dataIn =  pd.pivot_table(dataRaw, values='amount', index=['date'],
                    columns=['end_id'], aggfunc=np.sum, fill_value=0)
dataIn.head(1)

end_id,BL01,BL02,BL03,BL04,BL05,BL06,BL07,BL08,BL09,BL10,...,R20,R21,R22,R22A,R23,R24,R25,R26,R27,R28
date,,,,,,,,,,,,,,,,,,,,,
2017-01-01,6326,11444,5696,16834,12256,25559,45609,27781,18745,33240,...,5597,5937,15733,12194,2972,2067,9518,8439,6473,73538


In [100]:
dataOut =  pd.pivot_table(dataRaw, values='amount', index=['date'],
                    columns=['start_id'], aggfunc=np.sum, fill_value=0)
dataOut.head(1)

start_id,BL01,BL02,BL03,BL04,BL05,BL06,BL07,BL08,BL09,BL10,...,R20,R21,R22,R22A,R23,R24,R25,R26,R27,R28
date,,,,,,,,,,,,,,,,,,,,,
2017-01-01,6292,10268,5591,15834,12955,26456,42135,29606,18572,34387,...,5734,6801,14919,12373,2943,1957,8887,8603,6870,76751


In [101]:
#matrix = np.stack((dataOut.values, dataIn.values),-1)
matrix = dataOut.values + dataIn.values
matrix = matrix.astype(float)
matrix.shape

(638, 108)

In [102]:
# for i in range(matrix.shape[1]):
#         matrix[:, i] = (matrix[:, i] - np.mean(matrix[:, i])) / (np.std(matrix[:, i]))

In [103]:
for i in range(matrix.shape[0]):
        matrix[i,:] = (matrix[i,:] - np.mean(matrix[i,:])) / (np.std(matrix[i,:]))

In [104]:
stations = list(set(dataRaw.start_id))
n= len(stations)
# edge_index = [[a//n,a%n] for a in range(n*n)]
# edge_index = torch.tensor(edge_index, dtype=torch.long)

In [105]:
dates = list(dataOut.index)
DOW = list(pd.to_datetime(dataOut.index.values).dayofweek)
DOW = ((np.array(DOW) == 5) | (np.array(DOW) == 6)).astype(int)
DOW[:10]

array([1, 0, 0, 0, 0, 0, 1, 1, 0, 0])

In [106]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(input_dim, hiddenDim[0])
        self.linear2 = nn.Linear(hiddenDim[0], hiddenDim[1])
        self.linear3 = nn.Linear(hiddenDim[1], hiddenDim[2])
        self.linear4 = nn.Linear(hiddenDim[2], n_classes)

        self.init_weights(self.linear1)
        self.init_weights(self.linear2)
        self.init_weights(self.linear3)
        self.init_weights(self.linear4)
        
    def forward(self, x):
        #x, edge_index = data.x, data.edge_index
        x = self.linear1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.linear2(x)
        x = F.relu(x)
        x = self.linear3(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.linear4(x)
        return F.log_softmax(x, dim=1)
        
    def init_weights(self,m):
        if type(m) == nn.Linear:
            nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)
    
    def representation(self, x):
        x = self.linear1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.linear2(x)
        x = F.relu(x)
        x = self.linear3(x)
        return x
            

In [107]:
input_dim = matrix.shape[1]

In [108]:
hiddenDim = [500,100,40]
n_classes = 2

learning_rate=0.001
batch_size = 50
num_epochs = 100

In [109]:
# X_train, X_test, y_train, y_test = train_test_split(matrix, np.array(DOW), test_size=0.01, random_state=2019, shuffle=True)

In [110]:
matrixtensor = torch.tensor(matrix).float()
Y = torch.tensor(np.array(DOW), dtype=torch.long)

# X_train_Tensor = torch.tensor(X_train).float()
# X_test_Tensor = torch.tensor(X_test).float()
# y_train_Tensor = torch.tensor(y_train, dtype=torch.long)
# y_test_Tensor = torch.tensor(y_test, dtype=torch.long)


# X_train_Tensor = matrixtensor
# X_test_Tensor = matrixtensor
# y_train_Tensor = Y
# y_test_Tensor = Y


In [111]:
data_tensor = torch.utils.data.TensorDataset(matrixtensor, Y) 
dataloader = torch.utils.data.DataLoader(dataset = data_tensor, batch_size = batch_size, shuffle = True)

In [112]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
model = Net().to(device)
#data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

In [113]:
def validate():
    with torch.no_grad():
        predList = model(X_test_Tensor.cuda()).cpu().numpy()
        pred = np.argmax(predList,axis=1)    
        print('Val Acc: {:.4f}'.format( np.sum(pred == y_test) / pred.shape[0] ))

In [114]:
for epoch in range(num_epochs):
    for data in dataloader:
        X, y = data
        X = X.cuda()
        y = y.cuda()
#         print(X.shape)
#         print(y.shape)
        # ===================forward=====================
        output = model(X)
        #print(output.shape)
        loss = criterion(output, y)
        #MSE_loss = nn.MSELoss()(output, y)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
        .format(epoch + 1, num_epochs, loss.item()))
    #validate()

epoch [1/100], loss:0.7155
epoch [2/100], loss:0.6126
epoch [3/100], loss:0.5609
epoch [4/100], loss:0.5455
epoch [5/100], loss:0.5874
epoch [6/100], loss:0.5458
epoch [7/100], loss:0.5375
epoch [8/100], loss:0.5071
epoch [9/100], loss:0.3561
epoch [10/100], loss:0.4081
epoch [11/100], loss:0.4390
epoch [12/100], loss:0.2860
epoch [13/100], loss:0.2837
epoch [14/100], loss:0.3133
epoch [15/100], loss:0.2586
epoch [16/100], loss:0.3707
epoch [17/100], loss:0.2059
epoch [18/100], loss:0.3120
epoch [19/100], loss:0.1715
epoch [20/100], loss:0.2627
epoch [21/100], loss:0.3674
epoch [22/100], loss:0.3863
epoch [23/100], loss:0.1941
epoch [24/100], loss:0.2114
epoch [25/100], loss:0.2831
epoch [26/100], loss:0.1771
epoch [27/100], loss:0.1687
epoch [28/100], loss:0.1412
epoch [29/100], loss:0.2539
epoch [30/100], loss:0.2892
epoch [31/100], loss:0.1859
epoch [32/100], loss:0.1699
epoch [33/100], loss:0.3184
epoch [34/100], loss:0.1423
epoch [35/100], loss:0.4080
epoch [36/100], loss:0.0790
e

In [115]:
with torch.no_grad():
    predList = model(X_test_Tensor.cuda()).cpu().numpy()
    pred = np.argmax(predList,axis=1)    
    print('Val Acc: {:.4f}'.format( np.sum(pred == y_test) / pred.shape[0] ))

Val Acc: 0.0000


/home/urwa/miniconda3/envs/gcn/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  after removing the cwd from sys.path.


In [116]:
with torch.no_grad():
    representation = model.representation(matrixtensor.cuda()).cpu().numpy()
representation.shape

(638, 40)

In [117]:
# import events data
events_data =dataDir+'TaipeiEvents.csv'
df_events = pd.read_csv(events_data, encoding = "ISO-8859-1", parse_dates=['Date'], infer_datetime_format=True)


In [118]:
df_events.head()

,Type,Name,Date
0,National holiday,Republic Day/New Year's Day observed,2017-01-02
1,National holiday,Chinese New Year's Eve,2017-01-27
2,National holiday,Chinese New Year's Day,2017-01-28
3,National holiday,Chinese New Year Holiday 1,2017-01-29
4,National holiday,Chinese New Year Holiday 2,2017-01-30


In [119]:
holidayDates = df_events[df_events.Type == 'National holiday'].Date

In [120]:
holidayDates = [str(d.date()) for d in holidayDates]
dates = [str(d.date()) for d in dates]

In [121]:
anomalyIndex = [i for i,d in enumerate(dates) if d in holidayDates]
len(anomalyIndex)
indexBool = np.array([i in anomalyIndex for i in list(range(matrix.shape[0]))])

In [122]:
def anomalyDetection(y,pval = 0.2,iterN=5,n_com=1):
    #index of regular (non-outlier points)
    #rind=y[:,0]>-10 
    rind = np.array(range(y.shape[0]))
    
    #clustering model
    gm=GaussianMixture(n_components=n_com, n_init=100, max_iter=1000,random_state=0) 
    for i in range(iterN): #iterate
        print('Iteration {}'.format(i+1))  
        clustering=gm.fit(y[rind,:]) #fit EM clustering model excluding outliers
        l=clustering.score_samples(y) #estimate likelihood for each point
        Lthres=sorted(l)[int(len(l)*pval)] #anomaly threshold
        rind0=0+rind
        rind=l>Lthres #non-anomalous points
        if(sum(rind)==0):
            print('All anomalies in {} iterations'.format(i+1))
            break
        if all(rind==rind0):
            print('Convergence in {} iterations'.format(i+1))
            break
    return l < Lthres

In [123]:
def getResults(reducedMatrix,threshHolds,iterN=5,n_com=1):
    results = []
    for th in threshHolds:
        #th = thres/100
        print("Threshhold: ",th)
        outliers = anomalyDetection(reducedMatrix,th,iterN,n_com)

        tpr = sum(outliers & indexBool)/sum(indexBool)
        fpr = sum(outliers & ~indexBool)/sum(~indexBool)
        precision = sum(outliers & indexBool)/sum(outliers)

        F1 = 2 * (precision * tpr) / (precision + tpr)

        res = {'Cat':'Global', 'th':th, 'TPR':tpr, 'FPR':fpr, 'F1':F1, 'Precision':precision}
        results.append(res)

    resDf = pd.DataFrame(results)    
    return resDf

In [124]:
threshHolds = [0.01, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [125]:
Res1 = getResults(representation,threshHolds,iterN=5,n_com=1)

Threshhold:  0.01
Iteration 1
Iteration 2
Convergence in 2 iterations
Threshhold:  0.03
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.04
Iteration 1
Iteration 2
Convergence in 2 iterations
Threshhold:  0.05
Iteration 1
Iteration 2
Convergence in 2 iterations
Threshhold:  0.06
Iteration 1
Iteration 2
Convergence in 2 iterations
Threshhold:  0.07
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.08
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.1
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.2
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.3
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.4
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.5
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.6


In [126]:
Res1

,Cat,F1,FPR,Precision,TPR,th
0,Global,0.171429,0.004926,0.500000,0.103448,0.01
1,Global,0.375000,0.016420,0.473684,0.310345,0.03
2,Global,0.333333,0.026273,0.360000,0.310345,0.04
3,Global,0.366667,0.032841,0.354839,0.379310,0.05
4,Global,0.358209,0.042693,0.315789,0.413793,0.06
5,Global,0.410959,0.047619,0.340909,0.517241,0.07
6,Global,0.400000,0.057471,0.313725,0.551724,0.08
7,Global,0.347826,0.077176,0.253968,0.551724,0.10
8,Global,0.282051,0.172414,0.173228,0.758621,0.20
9,Global,0.236364,0.270936,0.136126,0.896552,0.30


In [127]:
from sklearn.decomposition import PCA
pca = PCA(n_components=20)
pca.fit(X_train)

PCA(copy=True, iterated_power='auto', n_components=20, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [128]:
reducedMatrixPCA = pca.transform(matrix)
print(reducedMatrixPCA.shape)

(638, 20)


In [129]:
Res1 = getResults(reducedMatrixPCA,threshHolds,iterN=5,n_com=1)

Threshhold:  0.01
Iteration 1
Iteration 2
Convergence in 2 iterations
Threshhold:  0.03
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.04
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.05
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.06
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.07
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.08
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.1
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.2
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.3
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.4
Iteration 1
Iteration 2
Iteration 3
Iteration 

In [130]:
Res1

,Cat,F1,FPR,Precision,TPR,th
0,Global,0.114286,0.006568,0.333333,0.068966,0.01
1,Global,0.458333,0.013136,0.578947,0.379310,0.03
2,Global,0.407407,0.022989,0.440000,0.379310,0.04
3,Global,0.400000,0.031199,0.387097,0.413793,0.05
4,Global,0.358209,0.042693,0.315789,0.413793,0.06
5,Global,0.383562,0.049261,0.318182,0.482759,0.07
6,Global,0.425000,0.055829,0.333333,0.586207,0.08
7,Global,0.391304,0.073892,0.285714,0.620690,0.10
8,Global,0.333333,0.165846,0.204724,0.896552,0.20
9,Global,0.254545,0.267652,0.146597,0.965517,0.30
